# Resource planning --- full

In [1]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [2]:
%load_ext kedro.ipython
%reload_kedro
%load_ext autoreload
%autoreload 2
%config IPCompleter.use_jedi=False
# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import logging

logging.basicConfig(level=logging.INFO)

import pickle

#import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

#import cufflinks as cf

# cf.go_offline(connected=True)
# cf.set_config_file(colorscale="plotly", world_readable=True)

import os
import sys

import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from GPSOdyssey import Kepler
from IPython.display import clear_output
from ipywidgets import fixed, interact, interact_manual, interactive
from keplergl import KeplerGl
from shapely import wkt

# sys.path.insert(0, '../../../mcarptif/')
# sys.path.insert(0, os.path.abspath('../../collection_diagnostics/'))
# sys.path.insert(0, os.path.abspath('../../OSM_processing/'))
import utils.process_gdf as process_gdf

# Extra options
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

from mcarptif.osmnx_network_extract.extract_grptif import NetworkExtract
from mcarptif.osmnx_network_extract.network_code import create_gdf, required_arc_plot
from mcarptif.solver.solve import solve_store_instance
from mcarptif.visualise.route_tables import RouteSummary
from utils.gdf_helpers import create_gdf


def df_style(df):
    if df[0] == "Total":
        return ["font-weight: bold"] * len(df)
    else:
        return [""] * len(df)

[06/28/23 10:32:51] INFO     Resolved project path as:                                              ]8;id=469548;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=7098;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/28/23 10:32:51] WARNING  /Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/importlib/__in ]8;id=338578;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=85504;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             it__.py:126: DeprecationWarning: `kedro.extras.datasets` is deprecated                
                             and will be removed in Kedro 0.19, install `kedro-datasets` instead by                
                             running `pip install kedro-datasets`.                                                 
                               return _bootstrap._gcd_import(name[level:], package, level)                         
                                                                                                                   

[06/28/23 10:32:52] INFO     NumExpr defaulting to 8 threads.                                          ]8;id=672021;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/numexpr/utils.py\utils.py]8;;\:]8;id=797434;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/numexpr/utils.py#160\160]8;;\

                    WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3 ]8;id=258277;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=35136;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             .10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely                  
                             GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS                       
                             version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions                    
                             between both will be slow.                                                            
                               warnings.warn(                                                                      
                                                                                                                   

[06/28/23 10:32:54] INFO     Kedro project Project RDI Cape Town                                    ]8;id=681196;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=669677;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

...

                    INFO     Resolved project path as:                                              ]8;id=77905;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=12790;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/28/23 10:32:54] INFO     Kedro project Project RDI Cape Town                                    ]8;id=883093;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=291512;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=733129;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=3888;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

                    WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3 ]8;id=634490;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=83391;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             .10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning:                                 
                                                                                                                   
                             Using slow pure-python SequenceMatcher. Install python-Levenshtein to                 
                             remove this warning                                                                   
                                                                                                                   
                                                                                                                   

[06/28/23 10:32:55] WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3 ]8;id=325574;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=67370;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             .10/site-packages/pyximport/pyximport.py:51: DeprecationWarning:                      
                                                                                                                   
                             the imp module is deprecated in favour of importlib and slated for                    
                             removal in Python 3.12; see the module's documentation for alternative                
                             uses                                                                                  
                                                                                                                   
                                                                                                                   

## Load network data

In [3]:
networkh5_file = "../data/05_model_input/sp_files/road_network_simplified_24645.h"
test_network = catalog.load("road_network_simplified_24645_edges")
test_network = test_network.drop(columns=["Unnamed: 0"], errors="ignore")
test_network_directed = catalog.load("road_network_simplified_24645_edges_directed")
nodes = catalog.load("road_network_simplified_24645_nodes")

                    INFO     Loading data from 'road_network_simplified_24645_edges'            ]8;id=401246;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=7779;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (GeoJSONDataSet)...                                                                   

[06/28/23 10:33:02] INFO     Loading data from 'road_network_simplified_24645_edges_directed'   ]8;id=652961;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=59733;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (GeoJSONDataSet)...                                                                   

[06/28/23 10:33:06] INFO     Loading data from 'road_network_simplified_24645_nodes'            ]8;id=869068;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=439077;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (GeoJSONDataSet)...                                                                   

In [4]:
test_network.shape

(22189, 19)

In [5]:
%%time
test_network = process_gdf.process_edges(test_network)
test_network.drop_duplicates(["geom_id"], inplace=True)
nodes.sp_index = nodes.index
test_network["arc_index"] = test_network.index
nodes["sp_index"] = nodes.index

[06/28/23 10:33:12] WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3 ]8;id=554882;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=682088;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             .10/site-packages/geopandas/geodataframe.py:205: UserWarning:                         
                                                                                                                   
                             Pandas doesn't allow columns to be created via a new attribute name -                 
                             see                                                                                   
                             https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-a                
                             ccess                                                                                 
                                                                                                                   
                                                                                                                   

CPU times: user 3.72 s, sys: 85.1 ms, total: 3.81 s
Wall time: 4.01 s


## Run scenarios

In [6]:
%%time
df_lat_lon_match = catalog.load("lat_lon_key_network_match_20220601_residential_streets_only")
df_producer_geo_all = catalog.load("syn_pop_scenarios_local_20220601")
infrastructure = catalog.load("infrastructure_sample_network_match")
solved = list(catalog.load("resource_summary_table_arcs_only_20220601"))

                    INFO     Loading data from                                                  ]8;id=958033;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=634611;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             'lat_lon_key_network_match_20220601_residential_streets_only'                         
                             (CSVDataSet)...                                                                       

                    INFO     Loading data from 'syn_pop_scenarios_local_20220601'               ]8;id=797699;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=275281;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from 'infrastructure_sample_network_match'            ]8;id=715907;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=441721;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (CSVDataSet)...                                                                       

                    INFO     Loading data from 'resource_summary_table_arcs_only_20220601'      ]8;id=835224;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=372160;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             (PartitionedDataSet)...                                                               

CPU times: user 82.3 ms, sys: 13.6 ms, total: 95.9 ms
Wall time: 106 ms


In [7]:
# keys_all = list(df_producer_geo_all)
# key = keys_all[0]

# routing_parameters = catalog.load(
#     "routing_parameters"
# )
# df_producer_geo = df_producer_geo_all[key]()
# df_producer_geo_waste = catalog.load("waste_gen_scenarios_local_20220601")[key]()

def assign_demand(df_producer_geo, df_lat_lon_match, df_producer_geo_waste, routing_parameters):
    df_producer_geo["lat_lon_id"] = (
        df_producer_geo[["parcelLat", "parcelLon"]]
        .round(8)
        .astype(str)
        .agg("-".join, axis=1)
    )
    df_producer_geo = df_producer_geo.merge(df_lat_lon_match, left_on="lat_lon_id", right_on="lat_lon_id").drop_duplicates(["arc_id"])

    df_producer_geo = df_producer_geo.merge(
        test_network[["arc_id", "geom_id", "geom_id_inv", "geom_id_order", "arc_index"]],
        validate="m:1",
    )
    df_producer_geo["land_use"] = df_producer_geo["mainDwellingType"].fillna("nan")


    total_waste = df_producer_geo_waste["waste"].sum()
    n_bins = len(df_producer_geo_waste)
    df_producer_geo_waste = df_producer_geo_waste.loc[df_producer_geo_waste["id"] != 0.01]
    df_producer_geo_waste = df_producer_geo_waste.loc[df_producer_geo_waste["id"].isin(df_producer_geo["id"])]
    routing_parameters.fillna(0, inplace=True)

    calculate = ["bin_service_cost"]
    df_producer_geo.drop(columns=calculate, inplace=True, errors="ignore")
    df_producer_geo["Domestic"] = np.ceil( n_bins * ( df_producer_geo["dist_collect"] / df_producer_geo["dist_collect"].sum()) ).astype(int)
    df_producer_geo["Non Domestic"] = 0
    df_producer_geo["H/M"] = 0
    df_producer_geo["category"] = "Landed"
    df_producer_geo["land_use"] = "Landed"
    df_producer_geo["total units"] =  np.ceil( n_bins * ( df_producer_geo["dist_collect"] / df_producer_geo["dist_collect"].sum()) ).astype(int)
    df_producer_geo["total_people"] =  np.ceil( n_bins * ( df_producer_geo["dist_collect"] / df_producer_geo["dist_collect"].sum()) ).astype(int)
    df_producer_geo["Postal Code"] = df_producer_geo["id"]

    df_producer_geo = df_producer_geo.merge(routing_parameters)
    df_producer_geo = df_producer_geo.merge(
        df_producer_geo_waste, how="left", validate="1:1"
    )
    df_producer_geo["demand"] = total_waste * ( df_producer_geo["dist_collect"] / df_producer_geo["dist_collect"].sum())
    df_producer_geo["bin_service_cost"] = df_producer_geo["bin_service_cost"] * df_producer_geo["total units"]
    return df_producer_geo

In [8]:
%reload_kedro
keys_all = list(df_producer_geo_all)
routing_parameters = catalog.load(
    "routing_parameters"
)
for keys in keys_all:
    if keys[0] == ".": continue
    if keys in solved: continue
    print(f"\n\n RUNNING SCENARIO {keys}")

    df_producer_geo = df_producer_geo_all[keys]()
    df_producer_geo_waste = catalog.load("waste_gen_scenarios_local_20220601")[keys]()
    df_producer_geo = assign_demand(df_producer_geo, df_lat_lon_match, df_producer_geo_waste, routing_parameters)
 
    # print("\n# offloads (12t)")
    # df_producer_geo.groupby("land_use")["demand"].sum() / 12000  # number of offloads
    # print("\n# units")
    # df_producer_geo.groupby("land_use")["total units"].sum()
    # print("\n# shifts (bin service cost)")
    # df_producer_geo.groupby("land_use")["bin_service_cost"].sum() / (
    #     3600 * 8
    # )  # number of shifts

    key_pois = infrastructure
    key_pois["Postal Code"] = key_pois["id"]

    key_pois["u"] = key_pois["arc_u"]
    key_pois["v"] = key_pois["arc_v"]
    key_pois["key"] = 0
    key_pois["geometry_point"] = key_pois["geometry"]
    key_pois["geometry"] = key_pois["geometry_arc"]
    key_pois = process_gdf.process_edges(key_pois)
    key_pois["geometry"] = key_pois["geometry_point"]
    key_pois = key_pois.merge(test_network[["arc_id", "arc_index"]], validate="1:1")

    offload = key_pois.loc[(key_pois["type"] != "depot")]

    key_depos = key_pois.loc[key_pois["type"] == "depot"]
    key_pois_list = list(key_depos["description"].unique())

    description = "Depot 2"

    key_depot_select = key_depos.copy()
    key_depot_select = key_depot_select.loc[key_depot_select["description"] == description]
    depot_arc_index = key_depot_select.iloc[0]["arc_index"]
    offload = key_pois.loc[key_pois["type"] != "depot"]

    descriptions = ["Offload 2", "Offload 3"]

    key_if_select = offload.copy()
    key_if_select = key_if_select.loc[key_if_select["description"].isin(descriptions)]
    if_arc_index = key_if_select["arc_index"].values

    key_list = list(df_producer_geo["category"].unique())

    descriptions = ["Landed"]

    bad_arcs = [
        "[(18.4571747, -33.9584372), (18.457166, -33.9585368), (18.457156, -33.9585885), (18.4571405, -33.9586324), (18.4571204, -33.9586875), (18.4570842, -33.9587648), (18.4568647, -33.9591331), (18.4568633, -33.9591415), (18.4568634, -33.9591465), (18.4568651, -33.9591518), (18.4568693, -33.9591571), (18.4568826, -33.9591636), (18.4570165, -33.9592207), (18.4570237, -33.9592212), (18.4570302, -33.9592194), (18.4570376, -33.9592149), (18.4570447, -33.9592066), (18.4570542, -33.9591921), (18.457325, -33.958752), (18.4573407, -33.9587173), (18.4573555, -33.9586744), (18.4573665, -33.9586219), (18.4573672, -33.9585957), (18.4573652, -33.958564), (18.4573568, -33.9585404), (18.4573451, -33.9585162), (18.457334, -33.9585017), (18.4573196, -33.9584906), (18.4573008, -33.9584812), (18.457278, -33.9584717), (18.4571747, -33.9584372)]"
    ]

    prod_select = df_producer_geo.copy()
    prod_select = prod_select.loc[~prod_select["geom_id_order"].isin(bad_arcs)]
    prod_select = prod_select.loc[prod_select["category"].isin(descriptions)]

    logging.info("Preparing road network.......\n")
    nodes["u"] = nodes["osmid"]
    network_info = NetworkExtract(
        test_network,
        test_network_directed,
        nodes,
        networkh5_file,
        round_cost=True,
        key_pois=key_pois,
    )
    
    depot = depot_arc_index
    ifs = if_arc_index
    req_arcs = network_info.arc_consolidation_standard(prod_select)
    network_info.load_required_arcs(req_arcs, merge_network=True)
    network_info.set_depot_arc(depot)
    network_info.set_if_arcs(ifs)
    network_info.loc_in_required_arcs()

    network_info.extend_required_inverse_arcs()

    network_info.check_main_list()
    network_info.load_distance_matrix()
    network_info.offload_calculations3D()

    network_info.network_gdf(prod_select)

    logging.info("Done!")

    cut_off = 100000
    bad_arcs = []
    for i in range(len(network_info.d)):
        if network_info.d[i, 0] > cut_off:
            bad_arc = network_info.reqArcList[i]
            bad_links = test_network.loc[test_network["arc_index"] == bad_arc]
            bad_arcs += bad_links["geom_id_order"].tolist()
    assert len(bad_arcs) == 0
    

    cut_off = 100000
    bad_arcs = []
    for i in range(len(network_info.d)):
        if network_info.d[0, i] > cut_off:
            bad_arc = network_info.reqArcList[i]
            bad_links = test_network.loc[test_network["arc_index"] == bad_arc]
            bad_arcs += bad_links["geom_id_order"].tolist()
    assert len(bad_arcs) == 0
    

    d_max = network_info.d.max()
    if d_max == np.inf:
        d_i = network_info.d.argmax(axis=0)[0]
        bad_arc = network_info.reqArcList[d_i]
        print(network_info.d[d_i, :])
        print(network_info.d[:, d_i])
        d_i2 = network_info.d[d_i, :].argmax()
        bad_arc2 = network_info.reqArcList[d_i2]
        print(network_info.d[d_i2, :])
        print(network_info.d[:, d_i2])
        test_network.loc[test_network["arc_index"] == bad_arc]
        test_network.loc[test_network["arc_index"] == bad_arc2]

    scenario_name = "Landed refuse"
    schedule_select = "weekly"
    offload_time = 15 * 60
    max_duration = 10.5 * 3600
    capacity = 10.5 * 1000
    travel_speed = 45 / 3.6
    service_speed = 30 / 3.6

    demand_increase_factor = {"daily": 1, "weekly": 1, "two_one": 1}
    increase_factor = demand_increase_factor[schedule_select]

    logging.info("Uploading scenario inputs...")
    prod_select_update = prod_select.copy()
    prod_arc_update = prod_select_update.copy()

    network_info.producer_demand = prod_arc_update
    prod_arc_update = network_info.add_demand_service_cost_to_ars(
        prod_select_update, service_speed
    )
    _ = pd.DataFrame(
        prod_arc_update.drop(columns=["arc_index"]).sum(numeric_only=True)
    ).astype(int)

    network_info.set_travel_speed(travel_speed)
    network_info.set_service_cost_and_demand(prod_arc_update)
    network_info.set_offload_time(offload_time)
    network_info.check_shapes()

    network_info.update_cost_matrix()
    network_info.update_offload_cost()

    network_info.set_vechile_capacity_constraint(capacity)
    network_info.set_vehicle_duration_constraint(max_duration)
    logging.info("Done")

    solution_df = solve_store_instance(
        "",
        improve=None,
        write_results=False,
        info=network_info,
        overwrite=True,
        test_solution=False,
        full_output=False,
        tollerance=60,
        nnFracLS=1,
        nnFracTS=0.25,
    )
    logging.info("Done generating routes")
    n_route = solution_df["route"].max() + 1
    print(f"Number of routes: {n_route}")

    logging.info("Preparing scenario outputs...")

    network_info.extend_prop_info()
    network_info.add_solution(solution_df, order=False)
    network_info.deconstruct_solution()

    network_info.vehicle_collection_schedule(schedule=schedule_select)
    network_info.extract_wide_offload_totals()

    network_info.add_traversal_time()
    network_info.add_time_formatted()
    network_info.add_constant_duration_time()

    _ = network_info.full_producer_report()

    route_summary = RouteSummary(network_info)
    route_sum_table = route_summary.route_summary()
    logging.info("Done preparing output")
    catalog.save("resource_summary_table_arcs_only_20220601", {keys: route_sum_table})

                    INFO     Resolved project path as:                                              ]8;id=753998;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=253118;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/28/23 10:33:13] INFO     Kedro project Project RDI Cape Town                                    ]8;id=440907;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=791748;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=477092;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=168671;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

                    INFO     Loading data from 'routing_parameters' (ExcelDataSet)...           ]8;id=833303;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=311777;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\



 RUNNING SCENARIO households_040


...

                    INFO     Preparing scenario outputs...                                         ]8;id=903680;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_34018/802359594.py\802359594.py]8;;\:]8;id=993494;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_34018/802359594.py#179\179]8;;\

[06/28/23 11:31:07] WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/src/mcarptif/visu ]8;id=216610;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=555381;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             alise/route_tables.py:219: FutureWarning:                                             
                                                                                                                   
                             The frame.append method is deprecated and will be removed from pandas                 
                             in a future version. Use pandas.concat instead.                                       
                                                                                                                   
                                                                                                                   

                    WARNING  /Users/ejwillemse/dev/projects/project-rdi-cape-town/src/mcarptif/visu ]8;id=9671;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=969086;file:///Users/ejwillemse/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             alise/route_tables.py:231: FutureWarning:                                             
                                                                                                                   
                             In a future version, `df.iloc[:, i] = newvals` will attempt to set the                
                             values inplace instead of always setting a new array. To retain the                   
                             old behavior, use either `df[df.columns[i]] = newvals` or, if columns                 
                             are non-unique, `df.isetitem(i, newvals)`                                             
                                                                                                                   
                                                                                                                   

                    INFO     Done preparing output                                                 ]8;id=119776;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_34018/802359594.py\802359594.py]8;;\:]8;id=645706;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_34018/802359594.py#196\196]8;;\

                    INFO     Saving data to 'resource_summary_table_arcs_only_20220601'         ]8;id=971915;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=884809;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#384\384]8;;\
                             (PartitionedDataSet)...                                                               